In [14]:
# This code is written by Yara Al-Shorman
# Date created: May 18 2022
# Last modified: May 22 2022
# Github repo: https://github.com/YaraAlShorman/Research-spring-22
# The purpose of this code is to predict the bulkflows for a large given set of galaxies (using machine learning)

### imports

In [15]:
# imports
import tensorflow as tf
from numpy import load, concatenate, expand_dims  # I am doing this to minimize unncessary imports, they take up too much RAM and time
from sklearn.preprocessing import MinMaxScaler
from math import floor
# import matplotlib.pyplot as plt

### data preprocessing

In [16]:
# stacking all of the data into one humungous tensor
# I am using tensors because they can be ragged and can be stacked
# these means we won't have to cut off our data to standardize array sizes
data = load(f'C:/Users/yaras/Documents/Research/Feldman/rotated-outerrim-cz-rand/rotated-0-error-40.npy.npz')
input_data = data['data']
scaler = MinMaxScaler()
scaler.fit(input_data)
input_data = scaler.transform(input_data)
input_data = tf.expand_dims(input_data, axis=0)
input_data = tf.RaggedTensor.from_tensor(input_data)
# if scaling the data this way doesn't work good enough, i can try scaling all the data at once, to one scale
# in this case, i am scaling every 'square' of data seprately
# i should try scaling the entire 'cube' at once

output_data = tf.convert_to_tensor(data['header'])
output_data = tf.expand_dims(output_data, axis=0)

num_of_files = 10  # variable for number of files to load, current limit is 3000

for i in range(1, num_of_files): # limit 3000
        array_data = load(f'C:/Users/yaras/Documents/Research/Feldman/rotated-outerrim-cz-rand/rotated-{i}-error-40.npy.npz')
        # 'data' data
        temp = tf.convert_to_tensor(array_data['data'], dtype='float64')
        scaler = MinMaxScaler()
        scaler.fit(temp)
        temp = scaler.transform(temp)
        temp = tf.expand_dims(temp, axis=0)  # dimensions have to be expandad to be able to concat along the outer-dimension 
        input_data = tf.concat([input_data, temp], axis=0)  # concat adds to existing dimensions, does not create new ones
        # 'input_data' is a tensor
        
        # 'header' data (bulkflows)
        temp = array_data['header']
        temp = expand_dims(temp, axis=0)
        output_data = concatenate([output_data, temp], axis=0)
        # 'output_data' is an array, because it is uniform
        # i scale it (normalize it) all at once, then i covnert it to a tensor
        
# next is scaling

In [17]:
# normalizing/scaling the data
scaler = MinMaxScaler()
scaler.fit(output_data)
output_data = scaler.transform(output_data)
output_data = tf.convert_to_tensor(output_data)

In [18]:
# splitting the data into train/test sections
# 75% -> train
# 25% -> test
train_percent = 0.75
input_train, input_test = (input_data[:floor(num_of_files * train_percent)], \
                                input_data[floor(num_of_files * train_percent)::])
output_train, output_test = (output_data[:floor(num_of_files * train_percent)], \
                                output_data[floor(num_of_files * train_percent)::])

# data is split and ready for the model

In [19]:
# tf.debugging.set_log_device_placement(True)

# # Place tensors on the CPU
# with tf.device('/GPU:0'):
#   a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
#   b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

# # Run on the GPU
# c = tf.matmul(a, b)
# print(c)

# print(tf.config.list_physical_devices())


### model creation

In [20]:
# optimizer

optimizer = tf.keras.optimizers.Adam()

In [21]:
# model creations

model0 = tf.keras.Sequential()
model0.add(tf.keras.layers.InputLayer(input_shape=[None, 4], ragged=True))
model0.add(tf.keras.layers.LSTM(512, dropout=0.25, activation='tanh'))
model0.add(tf.keras.layers.Dense(256, activation='relu'))
model0.add(tf.keras.layers.Dropout(0.5))
model0.add(tf.keras.layers.Dense(64, activation='relu'))
model0.add(tf.keras.layers.Dropout(0.5))
model0.add(tf.keras.layers.Dense(16, activation='relu'))
model0.add(tf.keras.layers.Dropout(0.5))
model0.add(tf.keras.layers.Dense(3, activation='linear'))

model0.compile(loss='mean_squared_error', optimizer=optimizer)

model0.fit(input_train, output_train, epochs=20, use_multiprocessing=True, workers=4)


Epoch 1/20


KeyboardInterrupt: 